# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
     # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS
            udacity
            
        WITH REPLICATION = {
             'class': 'SimpleStrategy'
            ,'replication_factor': 1
        }
    """)
    
except Exception as exception:
    print(exception)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
    
except Exception as exception:
    print(exception)

#### `DROP TABLE` statements are set
to clear keyspace objects when necessary

In [27]:

drop_table_session_activities = """
    DROP TABLE IF EXISTS
        udacity.session_activities
    ;
"""

drop_table_user_activities = """
    DROP TABLE IF EXISTS
        udacity.user_activities
    ;
"""

drop_table_song_audience = """
    DROP TABLE IF EXISTS
        udacity.song_audience
    ;
"""

drop_table_statements = list((drop_table_session_activities, drop_table_user_activities, drop_table_song_audience))

for statement in drop_table_statements:
    
    try:
        session.execute(statement)
        
    except Exception as exception:
        print(exception)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table_session_activites = """
    CREATE TABLE IF NOT EXISTS
        udacity.session_activities(        
             session_id INT
            ,item_in_session INT
            ,artist_name TEXT
            ,song_title TEXT
            ,song_length FLOAT        
            
            ,PRIMARY KEY(
                (session_id, item_in_session)
                ,artist_name
                ,song_title
                )
        )
    ;
"""

try:
    session.execute(create_table_session_activites)       
except Exception as exception:
    print(exception)

#### the `INSERT INTO` statement
is created for the `session_activities` table

In [29]:
insert_into_session_activities = """
    INSERT INTO
        udacity.session_activities(
             session_id
            ,item_in_session
            ,artist_name
            ,song_title
            ,song_length
        )
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""

In [30]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_into_session_activities, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [31]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_session_activities = """
    SELECT
         artist_name
        ,song_title
        ,song_length
    FROM
        udacity.session_activities
    WHERE
        session_id = 338
        AND
        item_in_session = 4
    ;        
"""

try:
    rows = session.execute(select_session_activities)       
except Exception as exception:
    print(exception)
    
for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### `CREATE TABLE` for the
`user_activities` table is defined below:

In [32]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table_user_activities = """
    CREATE TABLE IF NOT EXISTS
        udacity.user_activities(
             user_id INT
            ,session_id INT
            ,item_in_session INT
            ,user_first_name TEXT
            ,user_last_name TEXT
            ,artist_name TEXT
            ,song_title TEXT
            
            ,PRIMARY KEY(
                 (user_id, session_id)
                ,item_in_session
            )
        )
    ;
"""

try:
    session.execute(create_table_user_activities)       
except Exception as exception:
    print(exception)                    

#### `INSERT INTO user_activities` statement

In [33]:

insert_into_user_activities = """
    INSERT INTO
        udacity.user_activities(
             user_id
            ,session_id
            ,item_in_session
            ,user_first_name
            ,user_last_name
            ,artist_name
            ,song_title
        )
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""

In [34]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_into_user_activities, ( int(line[10])
                                                      ,int(line[8])
                                                      ,int(line[3])
                                                      ,line[1]
                                                      ,line[4]
                                                      ,line[0]
                                                      ,line[9]
                                                     )
                       )

In [35]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select_user_activities = """
    SELECT
         artist_name
        ,song_title
        ,user_first_name
        ,user_last_name
    FROM
        udacity.user_activities
    WHERE
        user_id = 10
        AND
        session_id = 182
    ;
"""

try:
    rows = session.execute(select_user_activities)       
except Exception as exception:
    print(exception)
    
for row in rows:
    print(row)                    

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


#### `CREATE TABLE song_audience` statement
the `song_audience` table will be used to answer query number 3:

In [36]:

create_table_song_audience = """
    CREATE TABLE IF NOT EXISTS
        udacity.song_audience(
                 song_title TEXT
                ,user_first_name TEXT
                ,user_last_name TEXT
                ,PRIMARY KEY(song_title,user_first_name, user_last_name)
            )
    ;
"""

try:
    session.execute(create_table_song_audience)
except Exception as exception:
    print(exception)

In [37]:

insert_into_song_audience = """
    INSERT INTO
        udacity.song_audience(
             song_title
            ,user_first_name
            ,user_last_name
        )
    VALUES(
         %s
        ,%s
        ,%s
    );
"""

In [38]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insert_into_song_audience, (line[9], line[1], line[4]))

In [39]:

select_song_audience = """
    SELECT
         song_title
        ,user_first_name
        ,user_last_name
    FROM
        udacity.song_audience
    WHERE
        song_title = 'All Hands Against His Own'
    ;
"""

try:
    rows = session.execute(select_song_audience)
except Exception as exception:
    print(exception)
    
for row in rows:
    print(row)

Row(song_title='All Hands Against His Own', user_first_name='Jacqueline', user_last_name='Lynch')
Row(song_title='All Hands Against His Own', user_first_name='Sara', user_last_name='Johnson')
Row(song_title='All Hands Against His Own', user_first_name='Tegan', user_last_name='Levine')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()